# Melbourne University AES/MathWorks/NIH Seizure Prediction

La epilepsia afecta a cerca del 1% de la población mundial y está caracterizada por la ocurrencia de convulsiones repentinas. Para la mayoría de los pacientes es suficiente proporcionarles una alta dosis de medicamentos anticonvulsivos para prevenir esta crisis, sin embargo por esta misma razón es que frecuentemente sufren de esfectos secundarios. Entre el 20 y 40 por ciento de los pacientes con epilepsia, los medicamentos no son efectivos.

Los sistemas de prediccción de convulsiones tienen el potencial de auxiliar a los pacientes con epilepsia a tener una mejor calidad de vida. En base a la actividad electrica del cerebro (EEG), un algoritmo efectivo debería de predecir con alto grado de probabilidad la ocurrencia de las convulsiones, por lo tanto se pueden diseñar dispositivos que adviertan al usuario y así evitar actividades peligrosas y administrar la dosis de medicamente adecuada.

![ ]("Seizure Prediction Grahic.png")

### Background

Existe nueva evidencia de que la actividad dinamica temporal del cerebro puede ser clasificada en 4 clases: Preictal, Interictal, Ictal y Post-ictal. Los sistemas de predicción deben de ser capaces de identificar un estado preictal que puede ser diferenciado de los demás estados, de esta manera se podría prevenir la convulsión. 

Actividad del cerebro humano fue guardada en la forma de EEG intracranial, lo que involucra electrodos posicionados en la superficie de la corteza cerebral capturando las señales electricas con un sistema de monitoreo.

### Dataset

Los datos están organizados en carpetas contenedoras de datos de entrenamiento y de pruebas por cada paciente. Para esta primera parte se selecciono la carpeta train_1 de datos de entrenammiento organizada en archivos nombrados de la siguiente manera.

i_j_k.mat - El j-ésimo segmento de datos correspondiente a la k clase (k = 0 para interictal, k = 1 para preictal) para el i-ésimo paciente.

Y dentro de este archivo .mat encontraremos lo siguiente: 

data: Una matriz de muestra de valores iEEG ordenada fila y columna como muestra y electrodo respectivamente.
nSamplesSegment: Número total de muestras (columnas).
iEEGsamplingRate: Tasa de muestreo de datos, es decir, el número de muestras de datos que representan 1 segundo de los EEG.
channelIndices: Un arreglo de índices correspondiente a las columnas en data.
sequence: El índice del segmento de datos dentro de la serie de una hora de clips.

![ ]("EEG.png")

In [1]:
import sys
import os
import numpy as np
import pandas as pd
from math import *
from scipy.io import loadmat

def mat_to_data(path):
    mat = loadmat(path)
    names = mat['dataStruct'].dtype.names
    ndata = {n: mat['dataStruct'][n][0, 0] for n in names}
    return ndata

In [ ]:
feat = mat_to_data('train_1/train_1/1_7_1.mat')
feat['data'] = np.vstack((feat['data'],mat_to_data('train_1/train_1/1_8_1.mat')['data']))
feat['data'] = np.vstack((feat['data'],mat_to_data('train_1/train_1/1_9_1.mat')['data']))
feat['data'] = np.vstack((feat['data'],mat_to_data('train_1/train_1/1_10_1.mat')['data']))
feat['data'] = np.vstack((feat['data'],mat_to_data('train_1/train_1/1_11_1.mat')['data']))
feat['data'] = np.vstack((feat['data'],mat_to_data('train_1/train_1/1_12_1.mat')['data']))

feat2 = mat_to_data('train_1/train_1/1_1_0.mat')
feat2['data'] = np.vstack((feat2['data'],mat_to_data('train_1/train_1/1_2_0.mat')['data']))
feat2['data'] = np.vstack((feat2['data'],mat_to_data('train_1/train_1/1_3_0.mat')['data']))
feat2['data'] = np.vstack((feat2['data'],mat_to_data('train_1/train_1/1_4_0.mat')['data']))
feat2['data'] = np.vstack((feat2['data'],mat_to_data('train_1/train_1/1_5_0.mat')['data']))
feat2['data'] = np.vstack((feat2['data'],mat_to_data('train_1/train_1/1_6_0.mat')['data']))

print(feat['data'])
print(feat2['data'])

In [ ]:
df = pd.DataFrame(data=feat['data'])
df2 = pd.DataFrame(data=feat2['data'])
for column in df:
    df.loc[df[column] == 0,column] = np.nan 
for column in df2:
    df2.loc[df2[column] == 0,column] = np.nan 
print(df.describe(),df2.describe(),sep='\n')

In [ ]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt

In [ ]:
df[0].plot(figsize=(20,10)).set_ylim([-300, 300])

In [ ]:
df2[0].plot(figsize=(20,10)).set_ylim([-300, 300])